# XGBoost training
This is an auto-generated notebook. To reproduce these results, attach this notebook to the **ML_cluster2** cluster and rerun it.
- Compare trials in the [MLflow experiment](#mlflow/experiments/3309376123777320/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Navigate to the parent notebook [here](#notebook/3309376123777319) (If you launched the AutoML experiment using the Experiments UI, this link isn't very useful.)
- Clone this notebook into your project folder by selecting **File > Clone** in the notebook toolbar.

Runtime Version: _10.3.x-cpu-ml-scala2.12_

In [0]:
import mlflow
import databricks.automl_runtime

# Use MLflow to track experiments
mlflow.set_experiment("/Users/ga107@duke.edu/databricks_automl/is_canceled_x_train_original1_2_csv-2022_03_21-02_01")

target_col = "is_canceled"

## Load Data
> **NOTE:** The dataset loaded below is a sample of the original dataset.
Stratified sampling using pyspark's [sampleBy](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameStatFunctions.sampleBy.html)
method is used to ensure that the distribution of the target column is retained.
<br/>
> Rows were sampled with a sampling fraction of **0.0461977505400177**

In [0]:
from mlflow.tracking import MlflowClient
import os
import uuid
import shutil
import pandas as pd

# Create temp directory to download input data from MLflow
input_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(input_temp_dir)


# Download the artifact and read it into a pandas DataFrame
input_client = MlflowClient()
input_data_path = input_client.download_artifacts("cbead4e4ff894c4aadfe60762f748c8e", "data", input_temp_dir)

df_loaded = pd.read_parquet(os.path.join(input_data_path, "training_data"))
# Delete the temp data
shutil.rmtree(input_temp_dir)

# Preview data
df_loaded.head(5)

,id,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,is_canceled
0,19,Resort Hotel,7,2015,July,27,1,0,4,2,...,0,No Deposit,250.0,NaN,0,Transient,153.00,0,1,0
1,37,Resort Hotel,36,2015,July,27,2,1,3,3,...,0,No Deposit,241.0,NaN,0,Transient,108.80,0,1,0
2,42,Resort Hotel,16,2015,July,27,2,2,3,2,...,0,No Deposit,NaN,NaN,0,Transient,123.00,0,0,0
3,63,Resort Hotel,93,2015,July,27,2,0,3,2,...,0,No Deposit,115.0,NaN,0,Contract,58.95,0,0,0
4,72,Resort Hotel,2,2015,July,27,3,0,2,2,...,0,No Deposit,3.0,NaN,0,Transient,91.50,0,0,0


### Select supported columns
Select only the columns that are supported. This allows us to train a model that can predict on a dataset that has extra columns that are not used in training.
`[]` are dropped in the pipelines. See the Alerts tab of the AutoML Experiment page for details on why these columns are dropped.

In [0]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
supported_cols = ["is_repeated_guest", "hotel", "company", "market_segment", "lead_time", "deposit_type", "adults", "days_in_waiting_list", "stays_in_week_nights", "distribution_channel", "required_car_parking_spaces", "children", "arrival_date_week_number", "customer_type", "arrival_date_month", "assigned_room_type", "arrival_date_year", "stays_in_weekend_nights", "arrival_date_day_of_month", "babies", "meal", "total_of_special_requests", "reserved_room_type", "agent", "id", "adr", "previous_bookings_not_canceled", "previous_cancellations", "country", "booking_changes"]
col_selector = ColumnSelector(supported_cols)

## Preprocessors

In [0]:
transformers = []

### Numerical columns

Missing values for numerical columns are imputed with mean for consistency

In [0]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

numerical_pipeline = Pipeline(steps=[
    ("converter", FunctionTransformer(lambda df: df.apply(pd.to_numeric, errors="coerce"))),
    ("imputer", SimpleImputer(strategy="mean"))
])

transformers.append(("numerical", numerical_pipeline, ["arrival_date_day_of_month", "days_in_waiting_list", "company", "arrival_date_week_number", "lead_time", "agent", "id", "adr", "previous_bookings_not_canceled", "stays_in_week_nights", "previous_cancellations"]))

### Categorical columns

#### Low-cardinality categoricals
Convert each low-cardinality categorical column into multiple binary columns through one-hot encoding.
For each input categorical column (string or numeric), the number of output columns is equal to the number of unique values in the input column.

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(handle_unknown="ignore")

transformers.append(("onehot", one_hot_encoder, ["is_repeated_guest", "hotel", "market_segment", "deposit_type", "adults", "distribution_channel", "required_car_parking_spaces", "children", "customer_type", "arrival_date_month", "assigned_room_type", "arrival_date_year", "stays_in_weekend_nights", "babies", "meal", "total_of_special_requests", "reserved_room_type", "country", "booking_changes"]))

In [0]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

### Feature standardization
Scale all feature columns to be centered around zero with unit variance.

In [0]:
from sklearn.preprocessing import StandardScaler

standardizer = StandardScaler()

## Train - Validation - Test Split
Split the input data into 3 sets:
- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters of the model)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [0]:
from sklearn.model_selection import train_test_split

split_X = df_loaded.drop([target_col], axis=1)
split_y = df_loaded[target_col]

# Split out train data
X_train, split_X_rem, y_train, split_y_rem = train_test_split(split_X, split_y, train_size=0.6, random_state=655295708, stratify=split_y)

# Split remaining data equally for validation and test
X_val, X_test, y_val, y_test = train_test_split(split_X_rem, split_y_rem, test_size=0.5, random_state=655295708, stratify=split_y_rem)

## Train classification model
- Log relevant metrics to MLflow to track runs
- All the runs are logged under [this MLflow experiment](#mlflow/experiments/3309376123777320/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Change the model parameters and re-run the training cell to log a different trial to the MLflow experiment
- To view the full list of tunable hyperparameters, check the output of the cell below

In [0]:
from xgboost import XGBClassifier

help(XGBClassifier)

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic', use_label_encoder: bool = True, **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of boosting rounds.
 |      use_label_encoder : bool
 |          (Deprecated) Use the label encoder from scikit-learn to encode the labels. For new
 |          code, we recommend that you set this parameter to False.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      learning_rate : Optional[float]
 |          Boosting learning rate (xgb's "eta")
 |      verbosity : Optional[int]
 |          The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 

In [0]:
import mlflow
import sklearn
from sklearn import set_config
from sklearn.pipeline import Pipeline

set_config(display="diagram")

xgbc_classifier = XGBClassifier(
  colsample_bytree=0.35753514112731505,
  learning_rate=0.16761400065767315,
  max_depth=4,
  min_child_weight=6,
  n_estimators=362,
  n_jobs=100,
  subsample=0.619136532641527,
  verbosity=0,
  random_state=655295708,
)

model = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
    ("classifier", xgbc_classifier),
])

# Create a separate pipeline to transform the validation dataset. This is used for early stopping.
pipeline = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
])

mlflow.sklearn.autolog(disable=True)
X_val_processed = pipeline.fit_transform(X_val, y_val)

model

Pipeline Pipeline(steps=[('column_selector',
 ColumnSelector(cols=['is_repeated_guest', 'hotel', 'company',
 'market_segment', 'lead_time',
 'deposit_type', 'adults',
 'days_in_waiting_list',
 'stays_in_week_nights',
 'distribution_channel',
 'required_car_parking_spaces', 'children',
 'arrival_date_week_number',
 'customer_type', 'arrival_date_month',
 'assigned_room_type', 'arrival...
 learning_rate=0.16761400065767315,
 max_delta_step=None, max_depth=4,
 min_child_weight=6, missing=nan,
 monotone_constraints=None, n_estimators=362,
 n_jobs=100, num_parallel_tree=None,
 predictor=None, random_state=655295708,
 reg_alpha=None, reg_lambda=None,
 scale_pos_weight=None,
 subsample=0.619136532641527, tree_method=None,
 validate_parameters=None, verbosity=0))]) ColumnSelector ColumnSelector(cols=['is_repeated_guest', 'hotel', 'company', 'market_segment',
 'lead_time', 'deposit_type', 'adults',
 'days_in_waiting_list', 'stays_in_week_nights',
 'distribution_channel', 'required_car_parking_spaces',
 'children', 'arrival_date_week_number', 'customer_type',
 'arrival_date_month', 'assigned_room_type',
 'arrival_date_year', 'stays_in_weekend_nights',
 'arrival_date_day_of_month', 'babies', 'meal',
 'total_of_special_requests', 'reserved_room_type', 'agent',
 'id', 'adr', 'previous_bookings_not_canceled',
 'previous_cancellations', 'country', 'booking_changes']) preprocessor: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('numerical',
 Pipeline(steps=[('converter',
 FunctionTransformer(func= at 0x7f47759e3940>)),
 ('imputer', SimpleImputer())]),
 ['arrival_date_day_of_month',
 'days_in_waiting_list', 'company',
 'arrival_date_week_number', 'lead_time',
 'agent', 'id', 'adr',
 'previous_bookings_no...
 OneHotEncoder(handle_unknown='ignore'),
 ['is_repeated_guest', 'hotel',
 'market_segment', 'deposit_type', 'adults',
 'distribution_channel',
 'required_car_parking_spaces', 'children',
 'customer_type', 'arrival_date_month',
 'assigned_room_type', 'arrival_date_year',
 'stays_in_weekend_nights', 'babies', 'meal',
 'total_of_special_requests',
 'reserved_room_type', 'country',
 'booking_changes'])]) numerical ['arrival_date_day_of_month', 'days_in_waiting_list', 'company', 'arrival_date_week_number', 'lead_time', 'agent', 'id', 'adr', 'previous_bookings_not_canceled', 'stays_in_week_nights', 'previous_cancellations'] FunctionTransformer FunctionTransformer(func= at 0x7f47759e3940>) SimpleImputer SimpleImputer() onehot ['is_repeated_guest', 'hotel', 'market_segment', 'deposit_type', 'adults', 'distribution_channel', 'required_car_parking_spaces', 'children', 'customer_type', 'arrival_date_month', 'assigned_room_type', 'arrival_date_year', 'stays_in_weekend_nights', 'babies', 'meal', 'total_of_special_requests', 'reserved_room_type', 'country', 'booking_changes'] OneHotEncoder OneHotEncoder(handle_unknown='ignore') remainder [['is_repeated_guest', 'hotel', 'company', 'market_segment', 'lead_time', 'deposit_type', 'adults', 'days_in_waiting_list', 'stays_in_week_nights', 'distribution_channel', 'required_car_parking_spaces', 'children', 'arrival_date_week_number', 'customer_type', 'arrival_date_month', 'assigned_room_type', 'arrival_date_year', 'stays_in_weekend_nights', 'arrival_date_day_of_month', 'babies', 'meal', 'total_of_special_requests', 'reserved_room_type', 'agent', 'id', 'adr', 'previous_bookings_not_canceled', 'previous_cancellations', 'country', 'booking_changes']] passthrough passthrough StandardScaler StandardScaler() XGBClassifier XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
 colsample_bynode=None, colsample_bytree=0.35753514112731505,
 enable_categorical=False, gamma=None, gpu_id=None,
 importance_type=None, interaction_constraints=None,
 learning_rate=0.16761400065767315, max_delta_step=None,
 max_depth=4, min_child_weight=6, missing=nan,
 monotone_constraints=None, n_estimators=362, n_jobs=100,
 num_parallel_tree=None, predictor=None, random_state=655

In [0]:
# Enable automatic logging of input samples, metrics, parameters, and models
mlflow.sklearn.autolog(log_input_examples=True, silent=True)

with mlflow.start_run(run_name="xgboost") as mlflow_run:
    model.fit(X_train, y_train, classifier__early_stopping_rounds=5, classifier__eval_set=[(X_val_processed,y_val)], classifier__verbose=False)
    
    # Training metrics are logged by MLflow autologging
    # Log metrics for the validation set
    xgbc_val_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_val, y_val, prefix="val_")

    # Log metrics for the test set
    xgbc_test_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_test, y_test, prefix="test_")

    # Display the logged metrics
    xgbc_val_metrics = {k.replace("val_", ""): v for k, v in xgbc_val_metrics.items()}
    xgbc_test_metrics = {k.replace("test_", ""): v for k, v in xgbc_test_metrics.items()}
    display(pd.DataFrame([xgbc_val_metrics, xgbc_test_metrics], index=["validation", "test"]))

/databricks/python/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


,precision_score,recall_score,f1_score,accuracy_score,log_loss,roc_auc_score,score
validation,0.960874,0.960938,0.960874,0.960938,0.129153,0.991787,0.960938
test,0.967693,0.967670,0.967680,0.967670,0.125907,0.990748,0.967670


In [0]:
# Patch requisite packages to the model environment YAML for model serving
import os
import shutil
import uuid
import yaml

None

import xgboost
from mlflow.tracking import MlflowClient

xgbc_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], str(uuid.uuid4())[:8])
os.makedirs(xgbc_temp_dir)
xgbc_client = MlflowClient()
xgbc_model_env_path = xgbc_client.download_artifacts(mlflow_run.info.run_id, "model/conda.yaml", xgbc_temp_dir)
xgbc_model_env_str = open(xgbc_model_env_path)
xgbc_parsed_model_env_str = yaml.load(xgbc_model_env_str, Loader=yaml.FullLoader)

xgbc_parsed_model_env_str["dependencies"][-1]["pip"].append(f"xgboost=={xgboost.__version__}")

with open(xgbc_model_env_path, "w") as f:
  f.write(yaml.dump(xgbc_parsed_model_env_str))
xgbc_client.log_artifact(run_id=mlflow_run.info.run_id, local_path=xgbc_model_env_path, artifact_path="model")
shutil.rmtree(xgbc_temp_dir)

## Feature importance

SHAP is a game-theoretic approach to explain machine learning models, providing a summary plot
of the relationship between features and model output. Features are ranked in descending order of
importance, and impact/color describe the correlation between the feature and the target variable.
- Generating SHAP feature importance is a very memory intensive operation, so to ensure that AutoML can run trials without
  running out of memory, we disable SHAP by default.<br />
  You can set the flag defined below to `shap_enabled = True` and re-run this notebook to see the SHAP plots.
- To reduce the computational overhead of each trial, a single example is sampled from the validation set to explain.<br />
  For more thorough results, increase the sample size of explanations, or provide your own examples to explain.
- SHAP cannot explain models using data with nulls; if your dataset has any, both the background data and
  examples to explain will be imputed using the mode (most frequent values). This affects the computed
  SHAP values, as the imputed samples may not match the actual data distribution.

For more information on how to read Shapley values, see the [SHAP documentation](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html).

In [0]:
# Set this flag to True and re-run the notebook to see the SHAP plots
shap_enabled = False

In [0]:
if shap_enabled:
    from shap import KernelExplainer, summary_plot
    # SHAP cannot explain models using data with nulls.
    # To enable SHAP to succeed, both the background data and examples to explain are imputed with the mode (most frequent values).
    mode = X_train.mode().iloc[0]

    # Sample background data for SHAP Explainer. Increase the sample size to reduce variance.
    train_sample = X_train.sample(n=min(100, len(X_train.index))).fillna(mode)

    # Sample a single example from the validation set to explain. Increase the sample size and rerun for more thorough results.
    example = X_val.sample(n=1).fillna(mode)

    # Use Kernel SHAP to explain feature importance on the example from the validation set.
    predict = lambda x: model.predict(pd.DataFrame(x, columns=X_train.columns))
    explainer = KernelExplainer(predict, train_sample, link="identity")
    shap_values = explainer.shap_values(example, l1_reg=False)
    summary_plot(shap_values, example, class_names=model.classes_)

## Inference
[The MLflow Model Registry](https://docs.databricks.com/applications/mlflow/model-registry.html) is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance. The snippets below show how to add the model trained in this notebook to the model registry and to retrieve it later for inference.

> **NOTE:** The `model_uri` for the model already trained in this notebook can be found in the cell below

### Register to Model Registry
```
model_name = "Example"

model_uri = f"runs:/{ mlflow_run.info.run_id }/model"
registered_model_version = mlflow.register_model(model_uri, model_name)
```

### Load from Model Registry
```
model_name = "Example"
model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
model.predict(input_X)
```

### Load model without registering
```
model_uri = f"runs:/{ mlflow_run.info.run_id }/model"

model = mlflow.pyfunc.load_model(model_uri)
model.predict(input_X)
```

In [0]:
# model_uri for the generated model
print(f"runs:/{ mlflow_run.info.run_id }/model")

runs:/db8d7eb6622f414f90ca663f5f0cb47c/model
